![](imgs/kodolamaczlogo.png)

# Przetwarzanie Big Data z użyciem Apache Spark

Autor notebooka: Jakub Nowacki.


## Podstawy Spark SQL - UDF

Podobnie jak w Hive czy wielu bazach danych, Spark SQL ma możliwość definiowania funkcji użytkownika, ang. User Defined Functions (UDF). Funkcje te biorą wartość z kolumny i przekształcają ją w inną wartość. 

In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
import pyspark.sql.functions as func
import pyspark.sql.types as types

spark = pyspark.sql.SparkSession.builder \
    .appName('udf') \
    .getOrCreate()

#sc = pyspark.SparkContext(appName='udf')
#sqlContext = pyspark.sql.SQLContext(sc)

Wygenerujmy najpierw dane:

In [3]:
import numpy as np
df = spark.createDataFrame([pyspark.sql.Row(kolumna=int(i)) for i in np.random.randint(0, 100, 100)])
df.printSchema()
df.show()

root
 |-- kolumna: long (nullable = true)

+-------+
|kolumna|
+-------+
|     99|
|     90|
|     56|
|     89|
|     15|
|     46|
|     85|
|     23|
|     47|
|      4|
|     61|
|     25|
|      3|
|     95|
|     27|
|     46|
|     43|
|     97|
|     96|
|     64|
+-------+
only showing top 20 rows



Do stworzenia funkcji używamy normalnej referencji do funkcji Python lub lambdy. Przykładowo, chcemy funkcję która zwróci klasyfikacje wartości:

In [4]:
def klasyfikuj(wartosc):
    return u'dużo' if wartosc > 50 else u'mało'

Powyższą funkcję należy teraz przekształcić w funkcje która działa na kolumnach. Robimy to używając funkcji `udf`:

In [7]:
klasyfikuj_udf = func.udf(klasyfikuj)
type(klasyfikuj_udf)

function

Tej funkcji można już użyć na kolumnie DataFrame, np możemy dodać kolumnę:

In [8]:
df.withColumn('ile?', klasyfikuj_udf('kolumna')).show()

+-------+----+
|kolumna|ile?|
+-------+----+
|     99|dużo|
|     90|dużo|
|     56|dużo|
|     89|dużo|
|     15|mało|
|     46|mało|
|     85|dużo|
|     23|mało|
|     47|mało|
|      4|mało|
|     61|dużo|
|     25|mało|
|      3|mało|
|     95|dużo|
|     27|mało|
|     46|mało|
|     43|mało|
|     97|dużo|
|     96|dużo|
|     64|dużo|
+-------+----+
only showing top 20 rows



Należu uważać z typami; jeżeli chcemy zwrócić inny typ niż tekstowy, należy przekazać tą informację w definicji funkcji UDF.

In [9]:
oblicz_udf = func.udf(lambda v: v + 123, returnType=types.IntegerType())

In [10]:
df.withColumn('oblicz', oblicz_udf('kolumna')).show()

+-------+------+
|kolumna|oblicz|
+-------+------+
|     99|   222|
|     90|   213|
|     56|   179|
|     89|   212|
|     15|   138|
|     46|   169|
|     85|   208|
|     23|   146|
|     47|   170|
|      4|   127|
|     61|   184|
|     25|   148|
|      3|   126|
|     95|   218|
|     27|   150|
|     46|   169|
|     43|   166|
|     97|   220|
|     96|   219|
|     64|   187|
+-------+------+
only showing top 20 rows



Aby wykorzystać funckje w zapytaniach SQL należy ją zarejestrować nieco inaczej, mianowicie:

In [ ]:
spark.udf.register('klasyfikuj', klasyfikuj, returnType=types.StringType())

In [ ]:
df.registerTempTable('df')
spark.sql('SELECT kolumna, klasyfikuj(kolumna) AS `ile?` FROM df').show()

## Dostęp do JVM

Spark używa [py4j](https://www.py4j.org/) aby wykonywać komendy na JVM. Dostęp do klas mamy w PySpark nieco ułatwiony, mianowicie używamy `_jvm` ze `SparkContext` podając pełną nazwę klasy z pakietem:

In [ ]:
s = spark.sparkContext._jvm.java.lang.String('tekst')
print(s, type(s))

Jak widać, py4j konwertuje typy. Możemy też użyć bardziej skomplikowanych obiektów:

In [ ]:
tokenizer = spark.sparkContext._jvm.java.util.StringTokenizer('Ala ma kota!')
print(tokenizer, type(tokenizer))

In [ ]:
while(tokenizer.hasMoreTokens()):
    print(tokenizer.nextToken())

Lub prościej:

In [ ]:
p = spark.sparkContext._jvm.java.util.regex.Pattern.compile('\\s')
print(p, type(p))
l = p.split('Ala ma kota!')
print(l, type(l))
list(l)

Oczywiście lepiej powyższą funkcjonalność wykorzystać do funkcji zwracających RDD lub DataFrame, które można użyć bezpośrednio w Pythonie.